# Day1 強化学習の位置付けを知る

## 環境

強化学習では，「環境」が与えられます．この環境の中を動き周りながらより良い行動パターンを学習していくのが強化学習です．

> 環境とは「行動」と行動に応じた「状態」の変化が定義されており，ある状態への到達に対し「報酬」が与えられる空間のことです．端的には，ゲームのようなものです．ゲームでは，ボタンを押したらキャラクターがジャンプしたりします．「ボタンを押す」のが行動であり，「キャラクターがジャンプする」のが状態の変化に相当します．そしてゴールに到達できれば「報酬」が得られます． *( p.15 )*

例えば $n \times m$ マスの２次元迷路の場合，状態は位置座標 $(i,j)$ に当たります．なので，状態集合 $S$ は次のように定義できます．

$$ S = \left\{ (i,j) \mid 0 \leq i < n, 0 \leq j < m \right\} $$

また，行動 $a$ は上下左右への遷移なので，行動集合 $A$ はこうなります．

$$ A = \left\{ a_U, a_D, a_L, a_R \right\} $$

ここで，$a_U$ は「上に移動」を表す行動です(UP の U．以下同様)．

ある状態 $s \in S$ から次の状態 $s' \in S$ に遷移するには行動 $a \in A$ を１つ選択する必要があります．このとき，その選び方にはなにか法則があると仮定しましょう．つまり，ある戦略に基づいて行動を決定するということです．この戦略を関数 $\pi$ として定義します．

$$ a = \pi(s) $$

$\pi$ は状態を入力にとり，行動を出力します．これで，次にとる行動 $a$ が計算できます．行動 $a$ を実行すると状態が遷移しますが，遷移先の状態は確率的に決まります．つまり，同じ状態で同じ行動をとったとしても，遷移先がいつも同じとは限らないのです．数学的に言うと，遷移先の状態 $s'$ が確立分布

$$　Pr(s_{t+1}=s' \mid s_t=s, a_t=a) = T(a,s)　$$

に従うということです．$Pr$ は，直前の状態($s_t$)が $s$，そこでとった行動($a_t$)が $a$ のとき，次の状態($s_{t+1}$)が $s'$ になる確率を表します．ここで，遷移先 $s'$ の確率分布は直前の状態 $s$ にのみ依存していると仮定しています．この確率分布を出力する関数 $T$ を遷移関数と呼びます．そういえば，行動 $a$ は戦略 $\pi$ によって決定されるので，$T(a,s) = T(\pi(s), s)$ となるわけですから，$T$ は $s$ にだけ依存します．なので，先の確率分布の式を次のように書き直すことにします．

$$ Pr(s_{t+1}=s' \mid s_t=s) = T(s) $$

あとは，この確率分布を元に次の状態を決定する関数があれば，状態 $s$ から $s'$ への遷移が計算できます．この関数を $\mathrm{choice}$ と定義することにします．

$$ s' = \mathrm{choice}(Pr) $$

これで状態 $s$ から $s'$ への遷移が計算できるようになりました．次は，この遷移の良し悪しを評価するための指標を考えます．これを即時報酬 $r$ と呼び，$r$ を計算する関数 $R$ を報酬関数と呼びます．即時報酬は直前の状態と遷移先にのみ依存します．

$$ r = R(s,s') $$

大事なのはこの即時報酬ではなく，環境の開始から終了までの期間に得られた即時報酬の総和です．この開始から終了までの期間を１エピソードと呼びます．「開始から終了までの期間」の定義は環境によって異なりますが，例えば２次元迷路の場合なら，スタートしてからゴールに辿り着くまでが１エピソードになります．**機械学習の目的とは，この１エピソードで得られる報酬の総和を最大化することにあります．**

### Python で実装

**numpy** を使います．

In [14]:
import numpy as np

$N \times M$ の２次元迷路を実装します．迷路は行列で表し，各要素の値はそれぞれ次の内のいずれかになるとします．
- **S** - Starting point
- **G** - Goal point
- **H** - Hole
- **F** - Frozen surface

`S`と`G`はそれぞれ１ずつ存在するものとします．`S`から出発して，`G`にたどり着けばゲームクリア，`H`に落ちればゲームオーバーです．`S`と`F`のマスは凍っているので，一定の確率でスリップし，望んだ方向とは別の方向に移動してしまいます．

In [15]:
grid = np.array([
    ['S','F','F','H'],
    ['F','F','F','F'],
    ['F','F','F','G'],
])

N = len(grid)
M = len(grid[0])

状態はタプルで表します．例えば，位置座標 $(0, 2)$ はそのまま `(0, 2)` と書けます．

行動はラムダ式で表します．状態を引数にとって，遷移後の状態を返すようにします．

In [16]:
a_up = lambda s: (max(0, s[0]-1), s[1])
a_down = lambda s: (min(s[0]+1, N-1), s[1])
a_left = lambda s: (s[0], max(0, s[1]-1))
a_right = lambda s: (s[0], min(s[1]+1, M-1))
A = [a_left, a_up, a_right, a_down]

次に戦略 $\pi$ ですが，ここではとりあえず，ランダムに行動を１つ選択するものとして定義します．利便性のために，行動(ラムダ式)本体ではなくインデックスを返すようにします．

In [17]:
def pi(s):
    return np.random.randint(len(A))

次は遷移関数です．出力である確率分布は，遷移確率行列として表します．この行列の $(i,j)$ 成分は，状態 $(i,j)$ に遷移する確率に対応しています．例えば今 $(1,1)$ にいて，行動 `a_up` を選択したとします．しかし，一定の確率（例えば $20\%$）でスリップするため，反対方向以外の方向（左または右）にそれぞれ $10\%$ の確率で移動してしまいます．この場合の遷移確率行列は次のようになります．

$$
Pr = \left(
    \begin{array}{cccc}
        0.0 & 0.8 & 0.0 & 0.0 \\
        0.1 & 0.0 & 0.1 & 0.0 \\
        0.0 & 0.0 & 0.0 & 0.0 
    \end{array}
\right)
$$

次のような場合は現在地に留まる確率が非０になります．例えば $(0,0)$ で行動 `a_up` を選択したとします．この場合 $80\%$ で上に，$10\%$ で左に移動しますが，上にも左にもこれ以上進めないので，どちらの場合も遷移先は $(0,0)$ になります．したがって $(0,0)$ の遷移確率は $80\%+10\%=90\%$ になります．

$$
Pr = \left(
    \begin{array}{cccc}
        0.9 & 0.1 & 0.0 & 0.0 \\
        0.0 & 0.0 & 0.0 & 0.0 \\
        0.0 & 0.0 & 0.0 & 0.0 
    \end{array}
\right)
$$

スリップする確率は `slip_prob` で指定します．デフォルトでは，望んだ方向に進める確率が約 $\frac{1}{3}$ になるように設定しています．

In [18]:
def T(s, policy=pi, slip_prob=0.66):
    pr = np.zeros((N, M))
    idx = policy(s)
    pr[A[idx](s)] += 1-slip_prob
    pr[A[(idx+1)%4](s)] += slip_prob/2
    pr[A[(idx-1)%4](s)] += slip_prob/2
    return pr

In [19]:
print(T((1,1), policy=lambda s: A.index(a_up), slip_prob=0.2))

[[0.  0.8 0.  0. ]
 [0.1 0.  0.1 0. ]
 [0.  0.  0.  0. ]]


次は $\mathrm{choice}$ 関数です．遷移確率行列 `pr` に従って，確率的に遷移先を選択します．

In [20]:
def choice(pr):
    r = np.random.rand()
    p_sum = 0.0
    for s, p in np.ndenumerate(pr):
        p_sum += p
        if r <= p_sum:
            return s # state

In [21]:
pr = T((1,1), policy=lambda s: A.index(a_up), slip_prob=0.2)
s = choice(pr)
print(pr)
print(s)

[[0.  0.8 0.  0. ]
 [0.1 0.  0.1 0. ]
 [0.  0.  0.  0. ]]
(0, 1)


最後は報酬関数です．ここでは単純に，各マスの種類に対して一定の報酬を与えることにします．

In [22]:
def R(s):
    if grid[s]=='G':
        return 1
    elif grid[s]=='H':
        return -1
    else:
        return 0

実際に動かしてみます．スタート`S`から出発して，ゴール`G`に辿り着くか穴`H`に落ちるまで繰り返し遷移します．

In [24]:
def next_step(s):
    s = choice(T(s))
    r = R(s)
    return s, r

steps = 0
reward = 0
s = next(filter(lambda s: s[1]=='S', np.ndenumerate(grid)), (0, 0))[0] # Find starting point
while grid[s] in ['S', 'F']:
    s, r = next_step(s)
    reward += r
    steps += 1
    
if grid[s] == 'G':
    print('GAME CLEAR!!')
else:
    print('GAME OVER...')

print('It took %d steps.'%steps)
print('Reward : %f'%reward)

GAME CLEAR!!
It took 55 steps.
Reward : 1.000000
